<a href="https://colab.research.google.com/github/DeltSystem32/CIS3187-Assignment/blob/master/CIS3187_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Business Intelligence Coursework: Neural Network
## By Daniel Anthony Cilia 315499(M)
---



## Project Imports



In [0]:
import pandas as pd

## Read and parse data

Obtain testing and traning data from two CSV files in the project repository on Github.

The data consists of 5-bit numbers being mapped to 3-bit via the Boolean function ¬ABC.

Converters are used to map values to string upon import in order to retain all leading zeros.

#### Training set:

In [234]:
training_data_url = "https://raw.githubusercontent.com/DeltSystem32/CIS3187-Assignment/master/training_data.csv?token=AIW66N5TCWXKRMEIPQPTPES6ENAXA"
training_df = pd.read_csv(training_data_url, converters={'input': lambda x: str(x), 'output': lambda x: str(x)})
training_df

,Num,input,output
0,1,00000,100
1,3,00010,100
2,4,00011,100
3,5,00100,101
4,7,00110,101
5,8,00111,101
6,9,01000,110
7,10,01001,110
8,11,01010,110
9,12,01011,110


#### Testing set:

In [235]:
testing_data_url = "https://raw.githubusercontent.com/DeltSystem32/CIS3187-Assignment/master/testing_data.csv?token=AIW66N6FBMAJSI6JR2GHI626ENAXE"
testing_df = pd.read_csv(testing_data_url, converters={'input': lambda x: str(x), 'output': lambda x: str(x)})
testing_df

,Num,input,output
0,2,00001,100
1,6,00101,101
2,13,01100,111
3,16,01111,111
4,23,10110,001
5,28,11011,010


#### Size of data:

In [236]:
print("Size of training set: " + str(len(training_df)))
print("Size of testing set: "+ str(len(testing_df)))

Size of training set: 26
Size of testing set: 6


## MLP Parameters

The Neural Network to be implemented will consist of the following:
*   **Number of input neurons**: 5
*   **Number of output neurons**: 3
*   **Number of hidden layers**: 1
*   **Number of hidden neurons**: 4
*   **Error threshold**: 0.2
*   **Learning rate**: 0.2
*   **Maximum number of epochs**: 999
*   **Transformation function**: Sigmoid

In [0]:
class parameters:
  NUM_IN_NEURONS = 5
  NUM_OUT_NEURONS = 3
  NUM_HIDDEN_LAYERS = 1
  NUM_HIDDEN_NEURONS = 4
  ERROR_THRESHOLD = 0.2
  LEARNING_RATE = 0.2
  MAX_EPOCHS = 999

### Sigmoid function implementation

The method will be called *transform()* so as to remain generic so that it can be 'hotswapped' with other transformation functions if the need arises.

In [0]:
 import math

 def transform(x):
    #sigmoid
    return 1 / (1 + math.exp(-x))
    

#add to MLP parameters
parameters.transform = transform

## MLP Class

### Initialisation

We will first begin by declaring the MLP's initialisation function. An array of 5-bit binary numbers is to be received:

In [0]:
class MLP:
  layers = None
  inputs = None
  outputs = None
  wH = None #hidden layer weights
  wO = None #output layer weights
    
  def __init__(self, input_df):
    wH = self.generate_weights(parameters.NUM_IN_NEURONS,parameters.NUM_HIDDEN_NEURONS)
    print('wh: ', wH)
    wO = self.generate_weights(parameters.NUM_HIDDEN_NEURONS,parameters.NUM_OUT_NEURONS)
    print("wO: ", wO)
    inputs = np.empty(shape=(1,parameters.NUM_IN_NEURONS))

    for index, row in input_df.iterrows():
      input = np.array(list(row["input"]),dtype=np.float64)
      output = np.array(list(row["output"]),dtype=np.float64)

      self.feedforward(wH, wO, input)

To initialise the weights, we will need to do so randomly. For this, a function is required to create a matrix of weights of any given size:

In [0]:
def generate_weights(self, rows, columns):
    return np.random.uniform(low=0,high=1, size=(rows,columns))

#add to MLP
MLP.generate_weights = generate_weights

### Feedforward

Function for applying the transformation function to all cells of a given array:

In [0]:
  def transform_array(self, net):
    return [parameters.transform(j) for j in net]

  # add to MLP
  MLP.transform_array = transform_array

Loop through hidden layer and output layer, transforming input into MLP output. Matrix multiplication is performed using numpy's dot function:

In [242]:
  #feed-forward
  def feedforward(self, wH, wO, inputs):
    print('inputs: ', inputs)
    netH = np.dot(inputs,wH)
    print(netH)
    outH = self.transform_array(netH)
    print("outH: ", outH)

    netO = np.dot(outH,wO)
    print(netO)
    outO = self.transform_array(netO)
    print("outO: ", outO)

#add to MLP
MLP.feedforward = feedforward

mlp = MLP(training_df)

wh:  [[0.01576605 0.5906287  0.76208551 0.97501824]
 [0.80866826 0.89854744 0.17784228 0.04529147]
 [0.65502128 0.17625196 0.42205184 0.27509772]
 [0.80442114 0.61966237 0.35965299 0.18806537]
 [0.80977049 0.09363837 0.81220825 0.64871731]]
wO:  [[0.82306544 0.01777181 0.34408909]
 [0.42681764 0.76436428 0.38777749]
 [0.94746826 0.72532501 0.96864975]
 [0.67954432 0.33006014 0.79251024]]
inputs:  [0. 0. 0. 0. 0.]
[0. 0. 0. 0.]
outH:  [0.5, 0.5, 0.5, 0.5]
[1.43844783 0.91876062 1.24651329]
outO:  [0.8082141741564391, 0.7147895064681612, 0.7766957108645218]
inputs:  [0. 0. 0. 1. 0.]
[0.80442114 0.61966237 0.35965299 0.18806537]
outH:  [0.6909194099930194, 0.6501417553910367, 0.5889564290132646, 0.5468782564236322]
[1.7758094  1.11691157 1.49374729]
outO:  [0.8551786395143379, 0.7534153944488953, 0.8166400537935685]
inputs:  [0. 0. 0. 1. 1.]
[1.61419163 0.71330074 1.17186123 0.83678269]
outH:  [0.833992526062239, 0.6711300921372539, 0.7634812776215959, 0.6977871803270931]
[2.17043218 1.31